<a href="https://colab.research.google.com/github/raven-gith/machinelearning1/blob/main/UTS/REGRESI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== INSTALL & IMPORT LIBRARY =====
!pip install gdown
import gdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold, mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# ===== 1. PENGUMPULAN DATA (gdown) =====
file_id = "1oEXSD137hPm_Fx5xSPRvlAr3silj3suK"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output="RegresiUTSTelkom.csv", quiet=False)

df = pd.read_csv("RegresiUTSTelkom.csv")

# ===== 2. PEMBERSIHAN DATA =====
df = df.drop_duplicates()
df = df.dropna()

Downloading...
From (original): https://drive.google.com/uc?id=1oEXSD137hPm_Fx5xSPRvlAr3silj3suK
From (redirected): https://drive.google.com/uc?id=1oEXSD137hPm_Fx5xSPRvlAr3silj3suK&confirm=t&uuid=5f63e404-ea46-4a6b-9f97-fc882661a772
To: /content/RegresiUTSTelkom.csv
100%|██████████| 443M/443M [00:08<00:00, 54.8MB/s]


In [ ]:
# ===== 3. FEATURE SELECTION =====
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

target = numerical_cols[-1]

# --- Constant / Quasi-Constant ---
selector = VarianceThreshold(threshold=0.01)
_ = selector.fit_transform(df[numerical_cols])

# --- Correlation ---
corr_matrix = df[numerical_cols].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop_corr = [column for column in upper.columns if any(upper[column] > 0.9)]
df.drop(columns=to_drop_corr, inplace=True)

# --- Mutual Information ---
mi = mutual_info_regression(df[numerical_cols[:-1]], df[target])
mi_series = pd.Series(mi, index=numerical_cols[:-1]).sort_values(ascending=False)
print("\nMutual Information Score (Top 10):\n", mi_series.head(10))

# === ANOVA (Hanya Disebut di Laporan, Skip Eksekusi untuk Hemat RAM) ===
print("\nANOVA dijelaskan secara teori karena dataset besar.")


Mutual Information Score (Top 10):
 10.20556     0.140863
68.40795     0.132764
-27.46348    0.109744
13.0162      0.094615
179.19498    0.089067
-7.26272     0.073053
-0.36994     0.072543
611.10913    0.069140
-8.41558     0.068148
6.02015      0.067854
dtype: float64

ANOVA dijelaskan secara teori karena dataset besar.


In [ ]:
# ===== 4. FEATURE ENGINEERING =====
df_encoded = df.copy()
for col in categorical_cols:
    df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col].astype(str))

scaler = StandardScaler()
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

# ===== 5. SPLIT DATA =====
X = df_encoded.drop(columns=[target])
y = df_encoded[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# ===== 6. MODELING =====
models = {
    'LinearRegression': LinearRegression(),
    'PolynomialRegression': Pipeline([('poly', PolynomialFeatures(degree=2)), ('linear', LinearRegression())]),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'KNN': KNeighborsRegressor(),
    'Bagging': BaggingRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'SVR': SVR()
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    results.append({'Model': name, 'MSE': mse, 'RMSE': rmse, 'R2': r2})

results_df = pd.DataFrame(results)
print("\nHasil Evaluasi Model:\n", results_df.sort_values(by='R2', ascending=False))

In [ ]:
# ===== 7. VISUALISASI =====
best_model_name = results_df.sort_values(by='R2', ascending=False).iloc[0]['Model']
print(f"\nModel Terbaik: {best_model_name}")

best_model = models[best_model_name]
y_pred_best = best_model.predict(X_test)

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred_best, alpha=0.7)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title(f'Actual vs Predicted Values ({best_model_name})')
plt.show()

SOAL ANALISA



1. Strategi Mengatasi Underfitting pada Linear Regression dan Decision Tree
Underfitting terjadi ketika model terlalu sederhana sehingga tidak dapat menangkap pola dari data.

Pendekatan A: Transformasi Fitur (Feature Engineering)

Contoh: menambahkan fitur polinomial (misalnya x², x³) pada Linear Regression.

Pengaruh terhadap bias-variance:

Menurunkan bias karena model bisa mempelajari pola non-linear.

Variance mungkin sedikit meningkat, tapi bisa dikontrol dengan regularisasi (Ridge/Lasso).

Cocok jika data memiliki hubungan non-linear namun masih cukup "teratur".

Pendekatan B: Ganti ke Model yang Lebih Kompleks

Contoh: ganti Linear Regression → Random Forest Regressor atau Gradient Boosting; Decision Tree → tambah ensemble (Bagging/Boosting).

Pengaruh terhadap bias-variance:

Bias berkurang drastis karena model dapat menangkap kompleksitas data.

Variance meningkat, namun ensemble seperti Bagging menstabilkan generalisasi.

Perbandingan:

Transformasi fitur efektif untuk model linear namun butuh eksplorasi manual.

Model kompleks otomatis menangani pola kompleks tetapi perlu tuning dan komputasi lebih.

2. Alternatif Loss Function untuk Regresi
a. MAE (Mean Absolute Error)
Keunggulan:

Lebih tahan terhadap outlier karena penalti linier.

Cocok jika kesalahan besar tidak harus dihukum keras.

Kelemahan:

Tidak terdiferensiasi di titik nol → sulit untuk optimasi berbasis turunan (gradient-based methods).

Cocok untuk: data dengan banyak outlier.

b. Huber Loss
Menggabungkan keunggulan MSE dan MAE: menggunakan MSE untuk kesalahan kecil dan MAE untuk kesalahan besar.

Keunggulan:

Lebih stabil untuk outlier dibanding MSE.

Lebih mudah dioptimasi dibanding MAE.

Kelemahan:

Memiliki hyperparameter delta yang harus ditentukan.

Cocok untuk: data dengan sebagian kecil outlier tapi tetap ingin mempertahankan sensitivitas terhadap error kecil.

Kesimpulan:

MSE → cocok untuk distribusi normal & penalti besar.

MAE → robust terhadap outlier.

Huber → tradeoff fleksibel antara keduanya.

3. Menilai Pentingnya Fitur Tanpa Mengetahui Nama
a. Koefisien Regresi (Linear Model)
Nilai absolut dari koefisien menunjukkan kontribusi relatif fitur (jika data sudah diskalakan).

Keterbatasan:

Hanya relevan jika fitur tidak berkorelasi (multikolinearitas merusak interpretasi).

Tidak menangkap interaksi non-linear.

b. Impurity-based Feature Importance (Tree-based Model)
Mengukur berapa banyak impurity (MSE reduction) yang dikurangi oleh suatu fitur pada Decision Tree/Random Forest.

Keterbatasan:

Bias terhadap fitur dengan banyak nilai unik.

Tidak stabil jika data terlalu kecil.

c. Permutation Importance
Ukur penurunan performa ketika satu fitur diacak → cocok untuk model apa pun.

Kelebihan: agnostik terhadap model.

Kekurangan: mahal secara komputasi dan bisa terpengaruh oleh fitur berkorelasi.

4. Desain Eksperimen untuk Tuning Hyperparameter
Langkah Umum:
Gunakan Grid Search atau Randomized Search dengan K-Fold Cross Validation.

Contoh:

max_depth untuk Decision Tree → [3, 5, 10, 20, None]

learning_rate untuk SGD → log scale dari 1e-4 sampai 1e-1

Trade-off Analysis:
Aspek	Grid Search	Randomized Search
Komputasi	Mahal, eksponensial jumlah kombinasi	Lebih hemat, cocok jika banyak parameter
Stabilitas	Lebih stabil, tapi overkill jika range luas	Sedikit fluktuatif, tapi cukup akurat
Generalisasi	K-Fold CV penting untuk uji stabil	Hindari overfitting dengan hold-out/test set

Gunakan Bayesian Optimization untuk efisiensi tinggi pada tuning kompleks.

5. Langkah Jika Residual Plot Menunjukkan Non-linear & Heteroskedastisitas
a. Tangani Non-Linearitas:
Transformasi Fitur: polinomial, log, root, atau Box-Cox.

Ganti Model: Gunakan model non-linear seperti Random Forest, Gradient Boosting, atau SVR.

b. Tangani Heteroskedastisitas:
Gunakan transformasi target (log, sqrt).

Model alternatif seperti Generalized Linear Models (GLM).

Gunakan Weighted Least Squares jika pola variansi bisa diestimasi.




PENJELASAN SETIAP MODEL


📊 Analisis dan Model Terbaik
Jika berdasarkan MSE, R², dan hasil validasi (belum terlihat langsung karena bagian evaluasi belum diekstrak sepenuhnya), maka biasanya:

Gradient Boosting Regressor cenderung terbaik dalam akurasi karena:

Mengurangi bias dan variance secara bertahap.

Bisa menangani hubungan non-linier kompleks.

Cocok untuk data tabular seperti ini.

Linear Regression atau KNN cenderung underfitting atau kurang stabil karena:

Tidak mampu menangkap non-linearitas (Linear Regression).

Sensitif terhadap distribusi dan outlier (KNN).